In [1]:
#include
import argparse
import os, imp, sys
import time
import logging
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
from datetime import datetime
from ast import literal_eval
from torchvision.utils import save_image
from torch.autograd import Variable
import copy, math
import numpy as np
import struct

In [2]:
#my script
sys.path.append('../')
from data import get_dataset, get_num_classes
from preprocess import get_transform
from utils import *
import models
import quantization

# Load Checkpoint

In [3]:
checkpoint_file = '/home/rusci/Work/quantized_neural_networks/results/Imagenet_ARM/mobilenetv2_test2/checkpoint.pth.tar'

In [4]:
if os.path.isdir(checkpoint_file):
#            results.load(os.path.join(checkpoint_file, 'results.csv'))
    checkpoint_file = os.path.join(
        checkpoint_file, 'model_best.pth.tar')
if os.path.isfile(checkpoint_file):
    checkpoint_loaded = torch.load(checkpoint_file)
    print('Model pretrained' )
else:
    print('here')

Model pretrained


In [5]:
checkpoint_loaded.keys()

dict_keys(['config', 'quantizer', 'val_prec1', 'model', 'epoch', 'fold_type', 'regime', 'add_config', 'best_prec1', 'state_dict'])

In [6]:
model_config = checkpoint_loaded['config']
activ_bits = model_config['activ_bits']
activ_type = model_config['activ_type']
dataset = model_config['dataset']
input_dim = model_config['input_dim']
input_size = input_dim
num_classes = model_config['num_classes']
type_quant = model_config['type_quant']
weight_bits = model_config['weight_bits']
width_mult = model_config['width_mult']
additional_config = ''
quant_add_config = checkpoint_loaded['add_config']
fold_type = checkpoint_loaded['fold_type']
quantizer_load = checkpoint_loaded['quantizer']
state_dict =  checkpoint_loaded['state_dict']

In [7]:
type_quant = model_config['type_quant']
weight_bits = model_config['weight_bits']
activ_bits = model_config['activ_bits']
fold_type = 'ICN'
quant_add_config = checkpoint_loaded['add_config']
dummy_input = torch.Tensor(1,3,224,224)
input_size = model_config['input_dim']
dataset = model_config['dataset']

In [8]:
input_size

224.0

# Generate fake-quantized MobilenetV2

In [9]:
model = models.mobilenetv2(activ_type='learned', act_bits=8)

In [ ]:
for m in model.module():

In [10]:
quantizer = quantization.QuantOp(model, quant_type = type_quant, weight_bits = weight_bits,bias_bits =32, \
                                 batch_fold_delay=0,batch_fold_type=fold_type, act_bits=activ_bits, \
                                 add_config=quant_add_config, dummy_input = dummy_input  )

Batch Folding:  False Batch Folding Delay:  0 Type ICN
<class 'models.imagenet.mobilenetv2.InvertedResidual'>
<class 'models.imagenet.mobilenetv2.InvertedResidual'>
<class 'models.imagenet.mobilenetv2.InvertedResidual'>
Supported activation layer but no method is here yet!
<class 'models.imagenet.mobilenetv2.InvertedResidual'>
<class 'models.imagenet.mobilenetv2.InvertedResidual'>
Supported activation layer but no method is here yet!
<class 'models.imagenet.mobilenetv2.InvertedResidual'>
Supported activation layer but no method is here yet!
<class 'models.imagenet.mobilenetv2.InvertedResidual'>
<class 'models.imagenet.mobilenetv2.InvertedResidual'>
Supported activation layer but no method is here yet!
<class 'models.imagenet.mobilenetv2.InvertedResidual'>
Supported activation layer but no method is here yet!
<class 'models.imagenet.mobilenetv2.InvertedResidual'>
Supported activation layer but no method is here yet!
<class 'models.imagenet.mobilenetv2.InvertedResidual'>
<class 'models.i

# Load Dataset

In [11]:
import torchvision.transforms as transforms

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

input_eval_transform = transforms.Compose([
        transforms.Scale(256),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        normalize
    ])
print(input_eval_transform)
val_data = get_dataset(dataset, 'val', input_eval_transform)
val_loader = torch.utils.data.DataLoader(
    val_data,
    batch_size=16, shuffle=False,
    num_workers=8, pin_memory=True)

Compose(
    Scale(size=256, interpolation=PIL.Image.BILINEAR)
    CenterCrop(size=(224, 224))
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)


/usr/local/lib/python3.5/dist-packages/torchvision/transforms/transforms.py:209: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


In [12]:
def forward(data_loader, model,  epoch=0, training=False, quantizer=None):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    end = time.time()
    model.eval()
    #model = model.cuda()
    
    max_i = 1
    min_i = -1
    bit_i = 8
    n_steps = (2**bit_i)-1
    eps = (max_i-min_i) / n_steps
    
    for i, (inputs, target) in enumerate(data_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        target = target.cuda(async=True)
        input_var = Variable(inputs.cuda(), volatile=not training)
        target_var = Variable(target)
        
        input_var = input_var.clamp(min_i,max_i).div(eps).round()
        
        if quantizer is not None:
            input_var = input_var.mul(eps)
            quantizer.store_and_quantize(training=False )

        # compute output
        output = model(input_var)
        if type(output) is list:
            output = output[0]
        values, indices = output.max(1)
        #print(indices, target)
        # measure accuracy and record loss
        prec1, prec5 = accuracy(output.data, target, topk=(1, 5))
        top1.update(prec1.item(), inputs.size(0))
        top5.update(prec5.item(), inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if quantizer is not None:
            quantizer.restore_real_value()

            
            
        if i % 100 == 0:
            print('{phase} - Epoch: [{0}][{1}/{2}]\t'
                         'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                         'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                         'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                         'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                             epoch, i, len(data_loader),
                             phase='TRAINING' if training else 'EVALUATING',
                             batch_time=batch_time,
                             data_time=data_time, top1=top1, top5=top5))

    print('Top1: ', top1.avg)
    return top1.avg, top5.avg




# Copy only clip_val from checkpoint

In [13]:
new_state_dict = {}
for item in state_dict.keys():
    if 'clip' in item:
        i = item.replace('module.','')
        print(i)
        new_state_dict[i] = state_dict[item]

features.0.2.clip_val
features.1.conv.0.2.clip_val
features.1.conv.3.clip_val
features.2.conv.0.2.clip_val
features.2.conv.1.2.clip_val
features.2.conv.4.clip_val
features.3.finalFakeQuant.clip_val
features.3.conv.0.2.clip_val
features.3.conv.1.2.clip_val
features.3.conv.4.clip_val
features.4.conv.0.2.clip_val
features.4.conv.1.2.clip_val
features.4.conv.4.clip_val
features.5.finalFakeQuant.clip_val
features.5.conv.0.2.clip_val
features.5.conv.1.2.clip_val
features.5.conv.4.clip_val
features.6.finalFakeQuant.clip_val
features.6.conv.0.2.clip_val
features.6.conv.1.2.clip_val
features.6.conv.4.clip_val
features.7.conv.0.2.clip_val
features.7.conv.1.2.clip_val
features.7.conv.4.clip_val
features.8.finalFakeQuant.clip_val
features.8.conv.0.2.clip_val
features.8.conv.1.2.clip_val
features.8.conv.4.clip_val
features.9.finalFakeQuant.clip_val
features.9.conv.0.2.clip_val
features.9.conv.1.2.clip_val
features.9.conv.4.clip_val
features.10.finalFakeQuant.clip_val
features.10.conv.0.2.clip_val
f

In [14]:
model.load_state_dict(new_state_dict, strict=False)

IncompatibleKeys(missing_keys=['features.0.0.weight', 'features.0.1.bias', 'features.0.1.running_mean', 'features.0.1.weight', 'features.0.1.running_var', 'features.1.conv.0.0.weight', 'features.1.conv.0.1.bias', 'features.1.conv.0.1.running_mean', 'features.1.conv.0.1.weight', 'features.1.conv.0.1.running_var', 'features.1.conv.1.weight', 'features.1.conv.2.bias', 'features.1.conv.2.running_mean', 'features.1.conv.2.weight', 'features.1.conv.2.running_var', 'features.2.conv.0.0.weight', 'features.2.conv.0.1.bias', 'features.2.conv.0.1.running_mean', 'features.2.conv.0.1.weight', 'features.2.conv.0.1.running_var', 'features.2.conv.1.0.weight', 'features.2.conv.1.1.bias', 'features.2.conv.1.1.running_mean', 'features.2.conv.1.1.weight', 'features.2.conv.1.1.running_var', 'features.2.conv.2.weight', 'features.2.conv.3.bias', 'features.2.conv.3.running_mean', 'features.2.conv.3.weight', 'features.2.conv.3.running_var', 'features.3.conv.0.0.weight', 'features.3.conv.0.1.bias', 'features.3.

In [15]:
model

MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LearnedClippedLinearQuantization(num_bits=4, clip_val=Parameter containing:
      tensor([2.7349], requires_grad=True), inplace)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): LearnedClippedLinearQuantization(num_bits=4, clip_val=Parameter containing:
          tensor([4.2259], requires_grad=True), inplace)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3):

# Compute S parameters

In [16]:
S_ai_l = [2/255]
Act_bits = []
for item in model.modules():
    if type(item) is models.linear_quantized_modules.LearnedClippedLinearQuantization:
        a_bits = item.num_bits
        Act_bits.append(a_bits)
        n_levels_a = (2**a_bits)-1
        a_max = item.clip_val.data.item()
        S_a = a_max / n_levels_a
        print('Learned',S_a,len(S_ai_l))
        S_ai_l.append(S_a)
    elif type(item) is models.linear_quantized_modules.LearnedClippedSymLinearQuantization:
        a_bits = item.num_bits
        Act_bits.append(a_bits)
        n_levels_a = (2**a_bits)-1
        a_max = item.clip_val.data.item()
        S_a = 2* a_max / n_levels_a
        print('Sym',S_a,len(S_ai_l))
        S_ai_l.append(S_a)
    elif type(item) is models.linear_quantized_modules.LearnedClippedSymLinearQuantizationADD:
        a_bits = item.num_bits
        #Act_bits.append(a_bits)
        n_levels_a = (2**a_bits)-1
        a_max = item.clip_val.data.item()
        S_a = 2* a_max / n_levels_a
        print('SymdADD',S_a,len(S_ai_l))
        S_ai_l.append(S_a)

Learned 0.18232577641805012 1
Learned 0.28172709147135416 2
Sym 0.019179263769411575 3
Learned 1.1951462427775066 4
Learned 0.22400767008463543 5
Sym 0.0210171755622415 6
SymdADD 0.024908028396905636 7
Learned 0.13129773139953613 8
Learned 0.923762321472168 9
Sym 0.0172790901333678 10
Learned 0.870475689570109 11
Learned 0.008250396391924689 12
Sym 0.01743301503798541 13
SymdADD 0.017899438446643306 14
Learned 0.004301923864028033 15
Learned 0.00505209202859916 16
Sym 0.011885747722550934 17
SymdADD 0.019913520065008426 18
Learned 0.0036952635821174175 19
Learned 0.004597809735466452 20
Sym 0.011167555229336608 21
Learned 0.004538429017160453 22
Learned 0.00627599463743322 23
Sym 0.013095423754523782 24
SymdADD 0.013931683933033663 25
Learned 0.0028008402562608907 26
Learned 0.003926560458014993 27
Sym 0.010910812078737744 28
SymdADD 0.01443641699996649 29
Learned 0.0028118154581855325 30
Learned 0.004286027422138289 31
Sym 0.0075239859375299195 32
SymdADD 0.015370744817397173 33
Learn

# The array below specify the S_ai and S_ao parameter of any given conv layer

In [18]:
S_ai_vector = [0,1,2,3,4,5,6,8,9,7,11,12,13,15,16,14,19,20,18,22,23,24,26,27,25,30,31,29,34,35,33,37,38,39,41,42,40,45,46,44,48,49,50,52,53,51,56,57,55,59,60,61,62]
S_ao_vector = [1,2,3,4,5,6,8,9,10,11,12,13,15,16,17,19,20,21,22,23,24,26,27,28,30,31,32,34,35,36,37,38,39,41,42,43,45,46,47,48,49,50,52,53,54,56,57,58,59,60,61,62]

## Residual Connection

In [20]:
S1 = [6,13,14,24,25,29,39,40,50,51]
S2 = [10,17,21,28,32,36,43,47,54,58]
S3 = [7,14,18,25,29,33,40,44,51,55]
nRes = len(S1)

S1_ai = [S_ai_l[S1[i]] for i in range(nRes)]
S2_ai = [S_ai_l[S2[i]] for i in range(nRes)]
S3_ai = [S_ai_l[S3[i]] for i in range(nRes)]
scaleRes = [ S1_ai[i]/S2_ai[i] for i in range(nRes)]
scaleRes2 =[ S2_ai[i]/S3_ai[i] for i in range(nRes)]


In [21]:
scaleRes

[1.2163357792581364,
 1.4667158890567393,
 1.602807246443911,
 1.200224480086433,
 1.8516360940471603,
 1.356381388616151,
 1.3496483922191647,
 1.1135775397862333,
 1.1742741069556837,
 0.8723022942097809]

In [22]:
weight_vector = []
bias_vector = []
M0_vector = []
N0_vector = []

# Computing Quantized values and M,N parameters

In [23]:
###################################          PARAMS BITS         #############################################
BIAS_BITS = 32  #INT32
M0_BITS = 32
N0_BITS = 8
BIAS_CH_BITS = 16
M0_BITS_LAST = 8
BIAS_FIXED_BITS = 0
    
for i_l,layer in enumerate(quantizer_load.param_to_quantize):
    quant_type = layer['quant_type']
    w_bits = layer['w_bits']
    conv_layer = layer['conv']
    bias_bits = layer['bias_bits']

    if quant_type == 'PerChannelsAsymMinMax':
        weight_org = conv_layer.weight.data.clone()
        w_min_tensor, w_max_tensor = 0,0

        weight_tensor, bias_tensor, w_min_mat, w_max_mat = quantizer._quant_PerChannelsAsymMinMax( \
                    conv_layer, layer['batch_norm'], layer['fold_type'], w_bits, i_l, w_min_tensor, \
                    w_max_tensor, training=False, get_quantized=False )
        
        conv_layer.weight.data.copy_(weight_org)
    else:
        print('Not recognized quantization scheme!')
        exit() 

###################################          LEAF layers          #############################################
    if type(conv_layer) is nn.Linear:
        n_levels_w = (2**w_bits)-1

        if quant_type == 'PerChannelsAsymMinMax':    # per channel quantization

            weight_quant_tensor = weight_tensor.clone()
            if bias_tensor is not False:
                bias_quant_tensor = bias_tensor
            else:
                bias_quant_tensor = False

            S_a_i = S_ai_l[S_ai_vector[i_l]]
            print(S_a_i)

            for v in range(weight_tensor.size(0)) :
                #w_min_value,w_max_value= layer['w_min_thr'].data[v], layer['w_max_thr'].data[v]
                w_min_value,w_max_value= weight_tensor[v].min(), weight_tensor[v].max()
                range_w = w_max_value-w_min_value
                if range_w == 0:
                    range_w = 1
                S_w = (range_w) / n_levels_w

                weight_quant_tensor[v] = weight_tensor[v].div(S_w).round()

                if bias_tensor is not False:
                    bias_quant_tensor[v] = bias_tensor[v].div( S_w * S_a_i ).mul(2**BIAS_FIXED_BITS).round().clamp(-2**(BIAS_BITS-1), 2**(BIAS_BITS-1)-1 ).div(2**BIAS_FIXED_BITS)

            w_max_value , _ = weight_tensor.reshape(weight_tensor.size(0), -1).max(1)
            w_min_value , _ = weight_tensor.reshape(weight_tensor.size(0), -1).min(1)

            range_w = w_max_value-w_min_value
            range_w.masked_fill_(range_w.eq(0), 1)
            S_w = (range_w) / n_levels_w
            M0, n_exp = quantizer._get_m0_nexp_vect(S_w)
            M0 = M0.mul(2**(M0_BITS_LAST-1)).round().clamp(-2**(M0_BITS_LAST-1),2**(M0_BITS_LAST-1)-1).div(2**(M0_BITS_LAST-1))   #assume Q.1.31
            n_exp = n_exp.clamp(-2**(N0_BITS-1),2**(N0_BITS-1)-1  )
            layer['quant_act'].M_ZERO = M0
            layer['quant_act'].N_ZERO = 2**n_exp    

            weight_vector.append(weight_quant_tensor)
            bias_vector.append(bias_quant_tensor)
            M0_vector.append(M0)
            N0_vector.append(2**n_exp)

            layer['quant_conv'].weight.data.copy_( weight_quant_tensor )
            if bias_quant_tensor is not False:
                layer['quant_conv'].bias = nn.Parameter(bias_quant_tensor, requires_grad = False)



###################################          OTHERS layers       #############################################
    else:
        S_a_i = S_ai_l[S_ai_vector[i_l]]
        S_a_o = S_ai_l[S_ao_vector[i_l]]
        print(S_a_i, S_a_o)

        n_levels_w = (2**w_bits)-1

        if quant_type == 'PerChannelsAsymMinMax':    # per channel quantization

            if layer['fold_type'] == 'ICN':

                batch_layer = layer['batch_norm']
                gamma_over_sigma, mu_tensor, beta_tensor = quantizer._get_BN_scale_factors(batch_layer)
                if bias_tensor is not False:
                    bias_tensor = bias_tensor.add(-mu_tensor)
                    print('hereeee')
                else:
                    bias_tensor = -mu_tensor.clone()

                #w_min_value,w_max_value= layer['w_min_thr'].data, layer['w_max_thr'].data
                w_max_value , _ = weight_tensor.reshape(weight_tensor.size(0), -1).max(1)
                w_min_value , _ = weight_tensor.reshape(weight_tensor.size(0), -1).min(1)
                range_w = w_max_value-w_min_value
                range_w.masked_fill_(range_w.eq(0), 1)
                S_w = (range_w) / n_levels_w
                bias_tensor = bias_tensor.add(beta_tensor/gamma_over_sigma).div(S_a_i*S_w).mul(2**BIAS_FIXED_BITS).round().clamp(-2**(BIAS_BITS-1),2**(BIAS_BITS-1)-1).div(2**BIAS_FIXED_BITS)  #assume INT8
                layer['quant_conv'].bias = nn.Parameter(bias_tensor, requires_grad = False)


                weight_quant_tensor = weight_tensor.clone()
                for v in range(weight_tensor.size(0)) :
                    #w_min_value, w_max_value= layer['w_min_thr'].data[v], layer['w_max_thr'].data[v]
                    w_min_value,w_max_value= weight_tensor[v].min(), weight_tensor[v].max()
                    range_w = w_max_value-w_min_value
                    if range_w == 0:
                        range_w = 1
                        print('hhhhh')

                    S_w = range_w / n_levels_w
                    weight_quant_tensor[v] = weight_tensor[v].div(S_w).round()
                    gamma_over_sigma[v] = gamma_over_sigma[v].mul((S_a_i*S_w)/S_a_o)

                weight_vector.append(weight_quant_tensor)
                bias_vector.append(bias_tensor)
                
                #print('min:', weight_quant_tensor.min(),'max: ', weight_quant_tensor.max())
                M0, n_exp = quantizer._get_m0_nexp_vect(gamma_over_sigma)
                M0 = M0.mul(2**(M0_BITS-1)).round().clamp(-2**(M0_BITS-1),2**(M0_BITS-1)-1).div(2**(M0_BITS-1))   #assume Q.1.31
                n_exp = n_exp.clamp(-2**(N0_BITS-1),2**(N0_BITS-1)-1  )
                
                M0_vector.append(M0)
                N0_vector.append(2**n_exp)
                #layer['quant_act'].M_ZERO = M0
                #layer['quant_act'].N_ZERO = 2**n_exp                    
                #layer['quant_act'].clip_val = nl_a_o
                #layer['quant_conv'].weight.data.copy_( weight_quant_tensor )

            else:
                print('Not supported yet!!')



0.00784313725490196 0.18232577641805012
0.18232577641805012 0.28172709147135416
0.28172709147135416 0.019179263769411575
0.019179263769411575 1.1951462427775066
1.1951462427775066 0.22400767008463543
0.22400767008463543 0.0210171755622415
0.0210171755622415 0.13129773139953613
0.13129773139953613 0.923762321472168
0.923762321472168 0.0172790901333678
0.024908028396905636 0.870475689570109
0.870475689570109 0.008250396391924689
0.008250396391924689 0.01743301503798541
0.01743301503798541 0.004301923864028033
0.004301923864028033 0.00505209202859916
0.00505209202859916 0.011885747722550934
0.017899438446643306 0.0036952635821174175
0.0036952635821174175 0.004597809735466452
0.004597809735466452 0.011167555229336608
0.019913520065008426 0.004538429017160453
0.004538429017160453 0.00627599463743322
0.00627599463743322 0.013095423754523782
0.013095423754523782 0.0028008402562608907
0.0028008402562608907 0.003926560458014993
0.003926560458014993 0.010910812078737744
0.013931683933033663 0.00

In [24]:
S_ai_l

[0.00784313725490196,
 0.18232577641805012,
 0.28172709147135416,
 0.019179263769411575,
 1.1951462427775066,
 0.22400767008463543,
 0.0210171755622415,
 0.024908028396905636,
 0.13129773139953613,
 0.923762321472168,
 0.0172790901333678,
 0.870475689570109,
 0.008250396391924689,
 0.01743301503798541,
 0.017899438446643306,
 0.004301923864028033,
 0.00505209202859916,
 0.011885747722550934,
 0.019913520065008426,
 0.0036952635821174175,
 0.004597809735466452,
 0.011167555229336608,
 0.004538429017160453,
 0.00627599463743322,
 0.013095423754523782,
 0.013931683933033663,
 0.0028008402562608907,
 0.003926560458014993,
 0.010910812078737744,
 0.01443641699996649,
 0.0028118154581855325,
 0.004286027422138289,
 0.0075239859375299195,
 0.015370744817397173,
 0.002456952076332242,
 0.007874408422731885,
 0.010643331677305932,
 0.0036586836272594976,
 0.006666722484663421,
 0.011649050432093003,
 0.012915984322043026,
 0.003575307247685451,
 0.005391867488038306,
 0.008631174237120386,
 0.0

# Define integer-only mobilenetV2

In [25]:
mobilenet_int = models.mobilenetv2(activ_type='learned', act_bits=8,integer=True)
mobilenet_int

MobileNetV2Integer(
  (features): Sequential(
    (0): ConvBNReLUInteger(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): ScaledClippedLinearQuantizationChannel(clip_val=1, inplace)
    )
    (1): InvertedResidualInteger(
      (conv): Sequential(
        (0): ConvBNReLUInteger(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): ScaledClippedLinearQuantizationChannel(clip_val=1, inplace)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): ScaledClippedLinearQuantizationChannelBiasSym(clip_val=[1,2], inplace)
      )
    )
    (2): InvertedResidualInteger(
      (conv): Sequential(
        (0): ConvBNReLUInteger(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): ScaledClippedLinearQuantizationChannel(clip_val=1, inplace)
        )
        (1): ConvBNReLUInteger(
          (0): Conv2d

# Copy quantized values into integer-only network

In [26]:
c = 0
for item in mobilenet_int.modules():
    if type(item) in [nn.Conv2d, nn.Linear]:
        item.weight.data.copy_( weight_vector[c] )
        item.bias = nn.Parameter(bias_vector[c] , requires_grad = False)
        c += 1

In [27]:
Act_bits
a = 0
m = 0
r = 0
M0_vector
N0_vector
for i in mobilenet_int.modules():
    if type(i) is models.imagenet.mobilenetv2.InvertedResidualInteger:
        if i.use_res_connect:
            print('*******',type(i))
            i.scaleRes = scaleRes[r] #this is still FP
            i.scaleRes2 =  scaleRes2[r] #this is still FP
            r+=1
    elif type(i) is models.imagenet.mobilenetv2.ScaledClippedLinearQuantizationChannel:
        print('----',type(i), Act_bits[a])
        a_bits = Act_bits[a]
        a +=1
        i.clip_val= 2**a_bits -1
        i.M_ZERO = M0_vector[m].cuda()
        i.N_ZERO = N0_vector[m].cuda()
        m+=1
    elif type(i) is models.imagenet.mobilenetv2.ScaledClippedLinearQuantizationChannelBiasSym:
        print('----',type(i), Act_bits[a])
        a_bits = Act_bits[a]
        i.clip_val_low= -2**(a_bits-1)
        i.clip_val_high=2**(a_bits-1) -1
        a +=1 
        i.M_ZERO = M0_vector[m].cuda()
        i.N_ZERO = N0_vector[m].cuda()
        m+=1            
    elif type(i) is models.imagenet.mobilenetv2.ScaledClippedLinearQuantizationChannelBias:
        print('----',type(i))
        i.M_ZERO = M0_vector[m].cuda()
        i.N_ZERO = N0_vector[m].cuda()
        m+=1       
    else:
        pass

---- <class 'models.linear_quantized_modules.ScaledClippedLinearQuantizationChannel'> 4
---- <class 'models.linear_quantized_modules.ScaledClippedLinearQuantizationChannel'> 4
---- <class 'models.imagenet.mobilenetv2.ScaledClippedLinearQuantizationChannelBiasSym'> 8
---- <class 'models.linear_quantized_modules.ScaledClippedLinearQuantizationChannel'> 2
---- <class 'models.linear_quantized_modules.ScaledClippedLinearQuantizationChannel'> 4
---- <class 'models.imagenet.mobilenetv2.ScaledClippedLinearQuantizationChannelBiasSym'> 8
******* <class 'models.imagenet.mobilenetv2.InvertedResidualInteger'>
---- <class 'models.linear_quantized_modules.ScaledClippedLinearQuantizationChannel'> 4
---- <class 'models.linear_quantized_modules.ScaledClippedLinearQuantizationChannel'> 2
---- <class 'models.imagenet.mobilenetv2.ScaledClippedLinearQuantizationChannelBiasSym'> 8
---- <class 'models.linear_quantized_modules.ScaledClippedLinearQuantizationChannel'> 2
---- <class 'models.linear_quantized_modu

# Test it !!!!

In [28]:
mobilenet_int

MobileNetV2Integer(
  (features): Sequential(
    (0): ConvBNReLUInteger(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): ScaledClippedLinearQuantizationChannel(clip_val=15, inplace)
    )
    (1): InvertedResidualInteger(
      (conv): Sequential(
        (0): ConvBNReLUInteger(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32)
          (1): ScaledClippedLinearQuantizationChannel(clip_val=15, inplace)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1))
        (2): ScaledClippedLinearQuantizationChannelBiasSym(clip_val=[-128,127], inplace)
      )
    )
    (2): InvertedResidualInteger(
      (conv): Sequential(
        (0): ConvBNReLUInteger(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1))
          (1): ScaledClippedLinearQuantizationChannel(clip_val=3, inplace)
        )
        (1): ConvBNReLUInteger(
          (0): Conv2d(96, 96, kernel_size=(3, 3), stride=(2, 2

In [29]:
mobilenet_int.eval()
forward(val_loader, mobilenet_int.cuda(), quantizer=None)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:22: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


EVALUATING - Epoch: [0][0/3125]	Time 0.885 (0.885)	Data 0.787 (0.787)	Prec@1 75.000 (75.000)	Prec@5 81.250 (81.250)
EVALUATING - Epoch: [0][100/3125]	Time 0.021 (0.057)	Data 0.000 (0.031)	Prec@1 43.750 (71.225)	Prec@5 75.000 (88.614)
EVALUATING - Epoch: [0][200/3125]	Time 0.033 (0.041)	Data 0.000 (0.016)	Prec@1 50.000 (62.065)	Prec@5 68.750 (83.364)
EVALUATING - Epoch: [0][300/3125]	Time 0.031 (0.057)	Data 0.000 (0.032)	Prec@1 87.500 (64.369)	Prec@5 93.750 (84.738)
EVALUATING - Epoch: [0][400/3125]	Time 1.030 (0.076)	Data 1.000 (0.050)	Prec@1 75.000 (63.622)	Prec@5 100.000 (84.133)
EVALUATING - Epoch: [0][500/3125]	Time 0.613 (0.089)	Data 0.586 (0.064)	Prec@1 81.250 (65.095)	Prec@5 93.750 (85.055)
EVALUATING - Epoch: [0][600/3125]	Time 0.021 (0.107)	Data 0.000 (0.082)	Prec@1 62.500 (63.467)	Prec@5 87.500 (84.599)
EVALUATING - Epoch: [0][700/3125]	Time 0.022 (0.116)	Data 0.000 (0.091)	Prec@1 43.750 (62.643)	Prec@5 87.500 (84.362)
EVALUATING - Epoch: [0][800/3125]	Time 0.413 (0.118)	Data

(56.546, 79.628)

In [30]:
weight_vector

[tensor([[[[  48.,  -16.,   54.],
           [ 119.,  -92.,   25.],
           [  38., -136.,  -54.]],
 
          [[  29.,  -22.,   55.],
           [  73., -119.,   -8.],
           [  41., -120.,  -29.]],
 
          [[ -92.,  -73.,  -36.],
           [ -41., -106.,  -55.],
           [ -96., -122.,  -88.]]],
 
 
         [[[ -12.,    7.,   17.],
           [  50.,   65.,    8.],
           [ -33.,  -68.,  -19.]],
 
          [[ -11.,   15.,    9.],
           [  74.,  121.,    9.],
           [ -68., -134.,  -29.]],
 
          [[  -3.,    8.,   20.],
           [  22.,   28.,    7.],
           [ -13.,  -42.,  -12.]]],
 
 
         [[[ -17.,   19.,    7.],
           [ -47.,   78.,  -27.],
           [ -72.,   93.,  -30.]],
 
          [[ -17.,   25.,    4.],
           [ -89.,  135.,  -49.],
           [-110.,  145.,  -47.]],
 
          [[  -5.,    4.,    7.],
           [ -22.,   39.,  -17.],
           [ -26.,   41.,  -20.]]],
 
 
         [[[  24.,  -15.,   -1.],
           [